In [143]:
from etr.auto_import import *
from etr.data.jquants.api import *
from etr.data.jquants.sqlite import apply_schema, upsert
from etr.config import Config
import sqlite3
from tqdm.auto import tqdm

ST = "2016-01-01"
ET = "2025-12-15"
DB_PATH = "../data/equity/jquants.sqlite" if not Config.JQUANTS_DB.startswith("/") else Config.JQUANTS_DB
apply_schema(db_path=DB_PATH)

## Token Refresh

In [144]:
mailaddress = Config.JQUANTS_ID
password = Config.JQUANTS_PW

# 1. Refresh Token取得 (1週間有効)
refresh_token = get_refresh_token(mailaddress, password)
# 2. ID Token取得 (24時間有効)
id_token = get_id_token(refresh_token)

## listed/info

In [109]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    info_list = get_listed_info(id_token, date=date.date())
    upsert(info_list, "listed_info", db_path=DB_PATH)

  0%|          | 0/436 [00:00<?, ?it/s]

## prices/daily_quotes

In [ ]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    df = get_price_daily_quotes(id_token=id_token, date=date.date())
    upsert(df, "prices_daily_quotes", db_path=DB_PATH)

  0%|          | 0/436 [00:00<?, ?it/s]

## fins/announcements

In [6]:
res = get_fins_announcement(id_token)
upsert(res, "fins_announcement", db_path=DB_PATH)

## fins/statements
---

In [49]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    res = get_fins_statements(id_token=id_token, date=date.date())
    upsert(res, "fins_statements", db_path=DB_PATH)

  0%|          | 0/436 [00:00<?, ?it/s]

## markets/trading_calendar

In [64]:
res = get_markets_trading_calendar(id_token=id_token)
upsert(res, "markets_trading_calendar", db_path=DB_PATH)

## markets/indices 指数

In [78]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    df = get_indices(id_token, date=date.date())
    upsert(df, "indices", db_path=DB_PATH)

## option/index_option 日経225オプション四本値 

In [90]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    df = get_index_option(id_token, date=date.date())
    upsert(df, "index_option", db_path=DB_PATH)

## markets/weekly_margin_interest 信用取引週末残高 

In [120]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    df = get_weekly_margin_interest(id_token=id_token, date=date.date())
    upsert(df, "markets_weekly_margin_interest", db_path=DB_PATH)

## markets/short_selling 業種別空売り比率 

In [122]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    df = get_short_selling(id_token=id_token, date=date.date())
    upsert(df, "markets_short_selling", db_path=DB_PATH)

## markets/short_selling_positions 空売り残高報告 

In [130]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    df = get_short_selling_positions(id_token=id_token, disclosed_date=date.date())
    upsert(df, "markets_short_selling_positions", db_path=DB_PATH)

## markets/daily_margin_interest 日々公表信用取引残高

In [131]:
for date in tqdm(pd.date_range(ST, ET, freq="B")):
    df = get_daily_margin_interest(id_token=id_token, date=date.date())
    upsert(df, "markets_daily_margin_interest", db_path=DB_PATH)

---

In [138]:
# with sqlite3.connect(DB_PATH) as con:
#     res = pd.read_sql("select * from markets_daily_margin_interest", con=con)